<img src="img/header.png"></img>

<h2> Menggunakan tweepy untuk Data Crawler (Misal, sbg Upaya Tanggap Bencana) </h2>

In [1]:
#!pip install preprocessor

In [2]:
#!pip install textblob

In [3]:
#!pip install nltk
#nltk.download("stopwords")
#nltk.download('punkt')

In [4]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
import nltk

In [5]:
#Twitter credentials for the app
consumer_key = "JEj5tRSA9JWjWV6imMOrUIVWV"        
consumer_secret = "7MEa00KHpUbjxble8pdlV74qPbvW2OHqLtjt45QQraJaAzRmAh"
access_key = "935208713551364097-W9Oy0IS2M1dRUQS5MZ6Dnz18BkHUP80"
access_secret = "jCANa7K7werTP2X1mnLlcRBFDHAJt9TZSCbC77FSNCj50"

In [6]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
#api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True) # untuk solve error "429 Too Many Requests response status code indicates the user has sent too many requests in a given amount of time ("rate limiting""

### Ide Analsis Data Twitter (sbg contoh)

<img src="img/ide-analisis.png" align="left"></img>

In [7]:
# Create file paths for the 3 CSV files
# declare file paths as follows for three files
gempa_tweets = "data/data_extraction/gempa_data.csv"
tsunami_tweets = "data/data_extraction/tsunami_data.csv"
banjir_tweets = "data/data_extraction/banjir_data.csv"

In [8]:
# Menentukan data nama-nama kolom untuk hasil ektraksi dari csv file
#COLS = ['id','created_at','source','original_text','clean_text','sentiment','polarity','subjectivity','lang',
#'favorite_count','retweet_count','original_author','possibly_sensitive','hashtags',
#'user_mentions','place','place_coord_boundaries']

COLS = ['id','created_at','source','original_text','clean_text','lang',
'favorite_count','retweet_count','original_author','hashtags',
'user_mentions','place','place_coord_boundaries']

<img src="img/data-field.png" align="left"></img>

In [9]:
#Declare HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [10]:
# Declare Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [11]:
# Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [12]:
# Combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [13]:
from nltk.corpus import stopwords
from nltk import word_tokenize

# Cleaning the data
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)

    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)


    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

In [14]:
#filecek=gempa_tweets
#print(filecek)
#os.path.exists(filecek)
#csvFile = open(filecek, 'a' ,encoding='utf-8')
#print(csvFile)

#filecek=tsunami_tweets
#print(filecek)
#os.path.exists(filecek)
#csvFile = open(filecek, 'a' ,encoding='utf-8')
#print(csvFile)

#filecek=banjir_tweets
#print(filecek)
#os.path.exists(filecek)
#csvFile = open(filecek, 'a' ,encoding='utf-8')
#print(csvFile)

In [15]:
#method write_tweets()
start_date='2019-01-31';
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        #df = pd.read_csv(file, header=0)
        try:
            df = pd.read_csv(file, header=0)
        except pd.io.common.EmptyDataError:
            print(file, " is empty and has been skipped.")
            df = pd.DataFrame(columns=COLS)
    else:
        df = pd.DataFrame(columns=COLS)
    
    csvFile = open(file, 'a' ,encoding='utf-8')
    
    #page attribute in tweepy.cursor and iteration
    #for page in tweepy.Cursor(api.search, q=keyword, \
                              #count=200, include_rts=False, since=start_date).pages(50):
    #for page in tweepy.Cursor(api.search, q=keyword,count=10, include_rts=False, since=start_date).pages():
    for status in tweepy.Cursor(api.search, q=keyword, include_rts=False, since=start_date, \
                                tweet_mode='extended').items(5):
        #for status in page:
        new_entry = []
        status = status._json

        ## check whether the tweet is in english or skip to the next tweet
        #if status['lang'] != 'id':
        #    print(status['lang'])
        #    continue

        #when run the code, below code replaces the retweet amount and
        #no of favorires that are changed since last download.
        print(status['created_at'])
        print(df['created_at'])
        if status['created_at'] in df['created_at'].values:  
            i = df.loc[df['created_at'] == status['created_at']].index[0]
            if status['favorite_count'] != df.at[i, 'favorite_count'] or \
               status['retweet_count'] != df.at[i, 'retweet_count']:
                df.at[i, 'favorite_count'] = status['favorite_count']
                df.at[i, 'retweet_count'] = status['retweet_count']
            continue


       #tweepy preprocessing called for basic preprocessing
        #clean_text = p.clean(status['text'])

        #call clean_tweet method for extra preprocessing
        #filtered_tweet=clean_tweets(clean_text)
        filtered_tweet=clean_tweets(status['full_text'])

        #pass textBlob method for sentiment calculations
        blob = TextBlob(filtered_tweet)
        Sentiment = blob.sentiment

        #seperate polarity and subjectivity in to two variables
        polarity = Sentiment.polarity
        subjectivity = Sentiment.subjectivity

        #new entry append
        #new_entry += [status['id'], status['created_at'],
        #              status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
        #              status['favorite_count'], status['retweet_count']]

        new_entry += [status['id'], status['created_at'],
                      status['source'], status['full_text'],filtered_tweet, status['lang'],
                      status['favorite_count'], status['retweet_count']]

        #to append original author of the tweet
        new_entry.append(status['user']['screen_name'])

        #try:
        #    is_sensitive = status['possibly_sensitive']
        #except KeyError:
        #    is_sensitive = None
        #new_entry.append(is_sensitive)

        # hashtagas and mentiones are saved using comma separted
        hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
        new_entry.append(hashtags)
        mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
        new_entry.append(mentions)

        #get location of the tweet if possible
        try:
            location = status['user']['location']
        except TypeError:
            location = ''
        new_entry.append(location)

        try:
            coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
        except TypeError:
            coordinates = None
        new_entry.append(coordinates)
        print(new_entry)
        single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
        df = df.append(single_tweet_df, ignore_index=True)
        print(df)
            #csvFile = open(file, 'a' ,encoding='utf-8')
    return df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [16]:
#declare keywords as a query untuk 3 kategori bencana
gempa_keywords = '#gempa OR #gempabumi OR #gempalaut OR #Gempa OR #GEMPA'
tsunami_keywords = '#tsunami OR #Tsunami OR #TSUNAMI OR #tsunamilaut OR #tsunamiawan'
banjir_keywords = '#banjir OR #Banjir OR #BANJIR OR #banjirbandang'

In [17]:
#call main method passing keywords and file path
write_tweets(gempa_keywords,  gempa_tweets)
write_tweets(tsunami_keywords, tsunami_tweets)
write_tweets(banjir_keywords, banjir_tweets)

Sun Jul 21 17:07:25 +0000 2019
0     Sun Jul 21 16:53:24 +0000 2019
1     Sun Jul 21 16:33:09 +0000 2019
2     Sun Jul 21 16:27:55 +0000 2019
3     Sun Jul 21 16:26:40 +0000 2019
4     Sun Jul 21 16:21:02 +0000 2019
5                         created_at
6     Sun Jul 21 16:53:24 +0000 2019
7     Sun Jul 21 16:33:09 +0000 2019
8     Sun Jul 21 16:27:55 +0000 2019
9     Sun Jul 21 16:26:40 +0000 2019
10    Sun Jul 21 16:21:02 +0000 2019
11    Sun Jul 21 17:07:25 +0000 2019
12                        created_at
13    Sun Jul 21 16:53:24 +0000 2019
14    Sun Jul 21 16:33:09 +0000 2019
15    Sun Jul 21 16:27:55 +0000 2019
16    Sun Jul 21 16:26:40 +0000 2019
17    Sun Jul 21 16:21:02 +0000 2019
18                        created_at
19    Sun Jul 21 16:53:24 +0000 2019
20    Sun Jul 21 16:33:09 +0000 2019
21    Sun Jul 21 16:27:55 +0000 2019
22    Sun Jul 21 16:26:40 +0000 2019
23    Sun Jul 21 16:21:02 +0000 2019
24    Sun Jul 21 17:07:25 +0000 2019
25    Sun Jul 21 16:09:01 +0000 2019
Name: c

Sun Jul 21 17:13:01 +0000 2019
0     Sun Jul 21 17:00:34 +0000 2019
1     Sun Jul 21 16:59:14 +0000 2019
2     Sun Jul 21 16:57:32 +0000 2019
3     Sun Jul 21 16:49:21 +0000 2019
4     Sun Jul 21 16:47:22 +0000 2019
5                         created_at
6     Sun Jul 21 17:00:34 +0000 2019
7     Sun Jul 21 16:59:14 +0000 2019
8     Sun Jul 21 16:57:32 +0000 2019
9     Sun Jul 21 16:49:21 +0000 2019
10    Sun Jul 21 16:47:22 +0000 2019
11    Sun Jul 21 17:05:16 +0000 2019
12    Sun Jul 21 17:04:57 +0000 2019
13    Sun Jul 21 17:04:27 +0000 2019
14                        created_at
15    Sun Jul 21 17:00:34 +0000 2019
16    Sun Jul 21 16:59:14 +0000 2019
17    Sun Jul 21 16:57:32 +0000 2019
18    Sun Jul 21 16:49:21 +0000 2019
19    Sun Jul 21 16:47:22 +0000 2019
20                        created_at
21    Sun Jul 21 17:00:34 +0000 2019
22    Sun Jul 21 16:59:14 +0000 2019
23    Sun Jul 21 16:57:32 +0000 2019
24    Sun Jul 21 16:49:21 +0000 2019
25    Sun Jul 21 16:47:22 +0000 2019
26    S

<img src="img/thumbs-up.png"></img>